### OneR algorithm

In [52]:
import pandas as pd
import numpy as np
import random

In [53]:
def generate_income():
    sums = [30, 40, 50]
    a = random.choice(sums)
    
    remaining_sums = [x for x in sums if x > a]
    remaining_sums.append(60)
    b = random.choice(remaining_sums)
    
    range = str(a) + 'K - ' + str(b) + 'K'
    return range


data = {
    'age': np.random.randint(20, 60, size=10),
    'gender': np.random.choice(['Male', 'Female'], size=10),
    'income_range': [generate_income() for _ in range(10)],
    'credit_card_insurance': np.random.choice([0, 1], size=10),
    'predicted_variable': np.random.choice([0, 1], size=10)
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the dataset
print(df)


   age  gender income_range  credit_card_insurance  predicted_variable
0   57    Male    50K - 60K                      1                   0
1   33  Female    30K - 50K                      1                   0
2   28  Female    30K - 60K                      0                   0
3   24  Female    40K - 50K                      1                   0
4   48    Male    50K - 60K                      1                   1
5   28    Male    40K - 60K                      1                   1
6   40  Female    30K - 60K                      0                   1
7   29    Male    50K - 60K                      0                   1
8   33  Female    50K - 60K                      1                   1
9   55    Male    50K - 60K                      0                   0


In [54]:
def get_nunique():
    nunique = []
    for column in df.columns:
        if column != 'predicted_variable':
            nunique.append((column, df[column].nunique()))
    
    return nunique

def binary_split(unique_values, column):
    to_be_sorted = []
    for value in unique_values:
        count = 0
        total = 0
        for index, row in df.iterrows():
            if row[column] == value:
                total += 1
                if row['predicted_variable'] == 1:
                    count += 1
        
        accuracy = round(count / total *100)
        if accuracy > 50:
            prediction = 1
        else:
            prediction = 0
        to_be_sorted.append((column, value, prediction))
        to_be_sorted.sort(key=lambda x: x[1])
    
    best_accuracy = 0
    best_range = []
    for i in range(len(to_be_sorted)-1):
        total_1 = 0
        count_1 = 0
        range_1 = to_be_sorted[:i+1]
        range_2 = to_be_sorted[i+1:]

        for item in range_1:
            total_1 += 1
            if item[2] == 1:
                count_1 += 1

        total_2 = 0
        count_2 = 0
        for item in range_2:
            total_2 += 1
            if item[2] == 0:
                count_2 += 1

        total = (count_1 + count_2) / len(to_be_sorted)
        
        if total > best_accuracy:
            best_accuracy = total
            best_range = len(range_1)    
    
    split = to_be_sorted[best_range][1]
    return split, best_accuracy

def oneR():
    nunique = get_nunique()
    overall_accuracies = []


    for item in nunique:
        column = item[0]
        unique_values = df[column].unique()
        overall_total = 0
        accuracies = []

        
        if item[1] < 7 or not isinstance(item[1], (int, float)):
            for value in unique_values:
                yes = []
                no = []
                cant_decide = []
                count = 0
                total = 0
                overall_total = 0

                for index, row in df.iterrows():
                    overall_total += 1
                    if row[column] == value:
                        total += 1
                        if row['predicted_variable'] == 1:
                            count += 1
                
                accuracy = round(count / total *100, 2)     
                accuracies.append((column, value, accuracy))

            for each in accuracies:
                if each[2] <= 50:
                    no.append(each)
                elif each[2] >= 100:
                    yes.append(each)
                else :
                    cant_decide.append(each)
            
            if len(yes) > len(no):
                no.extend(cant_decide)
            else:
                yes.extend(cant_decide)

            yes_updated = []
            for x, y, _ in yes:
                yes_updated.append((x, y, 1))

            no_updated = []
            for x, y, _ in no:
                no_updated.append((x, y, 0))

            decision = yes_updated + no_updated
            
            count = 0
            for column, value, prediction in decision:
                for index, row in df.iterrows():
                        if row[column] == value:
                            if row['predicted_variable'] == prediction:
                                count += 1
            overall_accuracies.append((column, round(count / overall_total, 2)))

        else :
            all_values = df[column].tolist()
            split_number, accuracy = binary_split(all_values, column)
            overall_accuracies.append((column, accuracy))
        
    print(overall_accuracies)
    overall_accuracies.sort(key=lambda x: x[1], reverse=True)
    model = overall_accuracies[0]
    print("model based on:", model)

oneR()


[('age', 0.6), ('gender', 0.6), ('income_range', 0.7), ('credit_card_insurance', 0.5)]
model based on: ('income_range', 0.7)
